## Preambule

In [1]:
import numpy as np
from tqdm import tqdm
from importlib import reload
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import scipy
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Run classes

Data reader class

In [3]:
import class_datareading
reload(class_datareading)
from class_datareading import datareading

datareader = datareading()
datareader.read_general()
datareader.read_ssps()
datareader.read_undata()
datareader.read_hdi()
datareader.read_historicalemis_jones()
datareader.read_ar6()
datareader.relation_budget_nonco2()
datareader.determine_global_nonco2_trajectories()
datareader.determine_global_budgets()
datareader.determine_global_co2_trajectories()
datareader.read_baseline()
datareader.read_ndc()
datareader.read_ndc_climateresource()
datareader.merge_xr()
datareader.add_country_groups()
datareader.save()
datareader.country_specific_datareaders()

# ==================================== #
# DATAREADING class                    #
# startyear:  2021
# ==================================== #
- Reading general data
- Reading GDP and population data from SSPs
- Reading UN population data and gapminder, processed by OWID (for past population)
- Read Human Development Index data
- Reading historical emissions (jones)
- Read AR6 data
- Get relationship between CO2 budgets and non-co2 reduction in 2050
- Computing global nonco2 trajectories
- Get global CO2 budgets
- Computing global co2 trajectories
- Reading baseline emissions
- Reading NDC data
- Reading NDC data from Climate resource
- Merging xrarray object
- Add country groups
- Save important files


In [63]:
import class_allocation
reload(class_allocation)
from class_allocation import allocation

allocator = allocation("USA", lulucf='incl', gas='GHG')
allocator.gf()
allocator.pc()
allocator.pcc()
allocator.pcb()
allocator.ecpc()
allocator.ap()
allocator.gdr()
allocator.save()

Allocations specifically for Norway and Netherlands if you want to harmonize to national data

In [4]:
import class_allocation
reload(class_allocation)
from class_allocation import allocation

for lulucf in ['incl', 'excl']:
    for gas in ['GHG', 'CO2']:
        for dataread_file in ['xr_dataread.nc', 'xr_dataread_nor.nc']: # This is only necessary if for a country a specific historical emissions profile is required beyond what is in Jones (e.g. Norway was requested)
            allocator = allocation('NOR', lulucf=lulucf, gas=gas, dataread_file=dataread_file)
            allocator.gf()
            allocator.pc()
            allocator.pcc()
            allocator.pcb()
            allocator.ecpc()
            allocator.ap()
            allocator.gdr()
            allocator.save()
            print('NOR', lulucf, gas, dataread_file, 'done')

NOR incl GHG xr_dataread.nc done
NOR incl GHG xr_dataread_nor.nc done
NOR incl CO2 xr_dataread.nc done
NOR incl CO2 xr_dataread_nor.nc done
NOR excl GHG xr_dataread.nc done
NOR excl GHG xr_dataread_nor.nc done
NOR excl CO2 xr_dataread.nc done
NOR excl CO2 xr_dataread_nor.nc done


Allocation class

In [7]:
import class_allocation
reload(class_allocation)
from class_allocation import allocation

regions_iso = np.load("K:/Data/Data_effortsharing/DataUpdate_ongoing/all_regions.npy", allow_pickle=True)
for cty in tqdm(['USA']):#regions_iso):
    allocator = allocation(cty, lulucf='incl', gas='GHG')
    allocator.gf()
    allocator.pc()
    allocator.pcc()
    allocator.pcb()
    allocator.ecpc()
    allocator.ap()
    allocator.gdr()
    allocator.save()

100%|██████████| 1/1 [02:01<00:00, 121.83s/it]


In [5]:
allocator.xr_alo

<xarray.Dataset> Size: 106MB
Dimensions:           (Scenario: 5, Region: 207, Time: 251, Temperature: 6,
                       Risk: 5, NonCO2red: 5, NegEmis: 5, Timing: 2,
                       Ambition: 2, Conditionality: 3, Hot_air: 2,
                       Convergence_year: 4)
Coordinates:
  * Scenario          (Scenario) <U4 80B 'SSP1' 'SSP2' 'SSP3' 'SSP4' 'SSP5'
  * Region            (Region) <U16 13kB 'AFG' 'ALB' ... 'Umbrella'
  * Time              (Time) int32 1kB 1850 1851 1852 1853 ... 2098 2099 2100
  * Temperature       (Temperature) float64 48B 1.5 1.6 1.7 1.8 1.9 2.0
  * Risk              (Risk) float64 40B 0.17 0.33 0.5 0.67 0.83
  * NonCO2red         (NonCO2red) float64 40B 0.17 0.33 0.5 0.67 0.83
  * NegEmis           (NegEmis) float64 40B 0.17 0.33 0.5 0.67 0.83
  * Timing            (Timing) <U9 72B 'Immediate' 'Delayed'
  * Ambition          (Ambition) <U3 24B 'max' 'min'
  * Conditionality    (Conditionality) <U13 156B 'conditional' ... 'unconditi...
  * Hot_air           (Hot_air) <U7 56B 'include' 'exclude'
  * Convergence_year  (Convergence_year) int32 16B 2040 2050 2080 2100
Data variables: (12/30)
    GDP               (Scenario, Region, Time) float64 2MB nan nan ... 1.436e+05
    Population        (Scenario, Region, Time) float64 2MB 3.753 3.768 ... 686.9
    GHG_hist          (Region, Time) float64 416kB 7.454 7.52 7.58 ... nan nan
    GHG_hist_excl     (Region, Time) float64 416kB 3.867 3.892 3.917 ... nan nan
    CO2_hist          (Region, Time) float64 416kB 3.521 3.561 3.597 ... nan nan
    CO2_hist_excl     (Region, Time) float64 416kB 0.0 0.0 0.0 ... nan nan nan
    ...                ...
    GHG_ndc_excl      (Region, Ambition, Conditionality) float64 10kB 33.69 ....
    GHG_ndc_excl_inv  (Region, Ambition, Conditionality) float64 10kB 42.28 ....
    GHG_ndc_excl_CR   (Region, Conditionality, Hot_air, Ambition) float64 20kB ...
    GF                (NegEmis, NonCO2red, Temperature, Risk, Timing, Time) float64 3MB ...
    PC                (Scenario, NegEmis, NonCO2red, Temperature, Risk, Timing, Time) float64 15MB ...
    PCC               (Time, Convergence_year, NegEmis, NonCO2red, Temperature, Risk, Timing, Scenario) float64 60MB ...

Allocation rules combining approaches (Robiou paper)

In [2]:
import class_allocation_combinedapproaches
reload(class_allocation_combinedapproaches)
from class_allocation_combinedapproaches import allocation_comb

allocator = allocation_comb(lulucf='excl', gas='GHG')
allocator.ecpc()
allocator.pc()
allocator.discounting_historical_emissions()
allocator.approach1gdp()
allocator.approach1hdi()
allocator.approach2()
allocator.approach2_transition()
allocator.combine()
allocator.get_relation_2030emis_temp()
allocator.determine_tempoutcomes()
allocator.save()

Temperature NDC-alignment metric

In [3]:
import class_tempalign
reload(class_tempalign)
from class_tempalign import tempaligning

tempaligner = tempaligning() # FIRST RUN AGGREGATOR FOR THIS!! (2030 alloc)
tempaligner.get_relation_2030emis_temp()
tempaligner.determine_tempoutcomes()
tempaligner.save()

# ==================================== #
# Initializing tempaligning class        #
# ==================================== #
- Determine relation between 2030-emissions and temperature outcome
- Determine temperature metric


100%|██████████| 6/6 [00:11<00:00,  1.96s/it]


- Save


Reading policy scenarios from ENGAGE

In [3]:
import class_policyscens
reload(class_policyscens)
from class_policyscens import policyscenadding

policyscenner = policyscenadding()
policyscenner.read_engage_data()
policyscenner.filter_and_convert()
policyscenner.add_to_xr()

# ==================================== #
# Initializing policyscenadding class  #
# ==================================== #
- Read ENGAGE scenarios and change region namings
- Filter correct scenarios and convert to xarray object
- Add to overall xrobject


Variance decomposition

In [10]:
import class_variancedecomp
reload(class_variancedecomp)
from class_variancedecomp import vardecomposing

vardecomposer = vardecomposing(startyear=2021, gas='GHG', lulucf='incl')
vardecomposer.sobolindices = {}
print('- Starting sobols for fixed years, over many countries')
timeseries = np.arange(2030, 2101, 5)
for year in timeseries:
    print('  Starting with', year)
    xr_cty, ar_time, array_dims, array_inputs, problem, samples = vardecomposer.prepare_global_sobol(year)
    vardecomposer.sobolindices[year] = vardecomposer.apply_decomposition(xr_cty, ar_time, array_dims, array_inputs, problem, samples)
vardecomposer.save(array_dims, timeseries)

# ==================================== #
# Initializing vardecomposing class    #
# ==================================== #
- Starting sobols for fixed years, over many countries
  Starting with 2030


207it [03:23,  1.02it/s]


  Starting with 2035


207it [02:43,  1.27it/s]


  Starting with 2040


...

  Starting with 2095


207it [02:26,  1.41it/s]


  Starting with 2100


207it [02:26,  1.42it/s]

- Save global results
